In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import pickle
import time
import os
import torch
from sklearn.preprocessing import normalize
import pandas as pd

In [2]:
aseg_2009_saliance_network = [18, 54, 11101, 11105, 11106, 11107, 11113, 11115, 11116, 11117, 11118, 11123, 11124, 11131, 12135, 11141, 11148, 11149, 11150, 11153,
                              11154, 11155, 11164, 11165, 11167, 11169, 11170, 12101, 12105, 12106, 12107, 12113, 12115, 12116, 12117, 12118, 12123, 12124,
                              12131, 12135, 12141, 12148, 12149, 12150, 12153, 12154, 12155, 12164, 12165, 12167, 11169, 12170]

aseg_saliance_network = [18, 54, 1002, 2002, 1003, 2003, 1012, 2012, 1014, 2014, 1027, 2027, 1035,  2035]

imp_roi = np.array([10,11,12,13,17,18,49,50,51,52,53,54,
            11101, 11102, 11104, 11105, 11106, 11107, 11108, 11109, 11111, 11112, 11113, 11114, 11115, 11116, 11117, 11118, 11119, 11120,
            11121, 11122, 11123, 11124, 11125, 11126, 11127, 11128, 11129, 11130, 11131, 11133, 11134, 11135, 11136, 11137, 11138, 11141, 11143,
            11144, 11145, 11146, 11147, 11148, 11149, 11150, 11151, 11153, 11154, 11155, 11157, 11158, 11159, 11160, 11161, 11162, 11164, 11165, 11166, 11167,
            11168, 11169, 11170, 11171, 11172, 11173, 11174, 
            12101, 12102, 12104, 12105, 12106, 12107, 12108, 12109, 12111, 12112, 12113, 12114, 12115, 12116, 12117, 12118, 12119, 12120,
            12121, 12122, 12123, 12124, 12125, 12126, 12127, 12128, 12129, 12130, 12131, 12133, 12134, 12135, 12136, 12137, 12138, 12141, 12143, 12144,
            12145, 12146, 12147, 12148, 12149, 12150, 12151, 12153, 12154, 12155, 12157, 12158, 12159, 12160, 12161, 12162, 12164, 12165, 12166, 12167,
            12168, 12169, 12170, 12172, 12173, 12174])
  
sub_det_df = pd.read_excel('data/rs_fmri_sub_details_WO_AS.xlsx')
cur_sub_list_before = sub_det_df['Subject_ID_before'].to_numpy()
cur_sub_list_after = sub_det_df['Subject_ID_after'].to_numpy()
subject_ids = np.concatenate((cur_sub_list_before,cur_sub_list_after))
labels = sub_det_df['labels'].to_numpy()
print('# of subjects to Process:', len(subject_ids))    

sub_to_label_dict = {}
for i in range(len(cur_sub_list_before)):
    sub_to_label_dict[cur_sub_list_before[i]] = labels[i]
    
for i in range(len(cur_sub_list_after)):
    sub_to_label_dict[cur_sub_list_after[i]] = labels[i]
    

# of subjects to Process: 178


In [3]:
data_dir = '/data-braingut/bharath/fmri_pre_process_native_space/BP_3PC_anaticor_motreg_noscale_4blur'
data_dir1 = '/data-braingut/bharath/fmri_bandpass'

In [4]:
def extract_conn_hist(cur_sub,imp_roi, data_dir, data_dir1):
    brik1 = nib.load(('%s/%s/errts.%s.fanaticor+orig.BRIK')%(data_dir,cur_sub,cur_sub))
    fmri_img1 = brik1.get_fdata()
    file_path_seg = ('%s/%s/follow_ROI_aeseg+orig.BRIK.gz')%(data_dir1,cur_sub)
    brik_seg = nib.load(file_path_seg)    
    seg_img = np.squeeze(brik_seg.get_fdata())
    
    mask_brik = nib.load('%s/%s/full_mask.%s+orig.BRIK.gz'%(data_dir1,cur_sub,cur_sub))
    mask_img = np.squeeze(mask_brik.get_fdata())
    seg_img[mask_img == 0] = 0    

    mean_roi_dict = {}
    average_mat = np.zeros((fmri_img1.shape[3],len(imp_roi)))
    for i,rois in enumerate(imp_roi):
        roi_bold = fmri_img1[seg_img == rois,:].transpose()
        average_mat[:,i] = np.mean(roi_bold, axis = 1)

    average_mat = normalize(average_mat, axis = 0)   
    
    data = torch.from_numpy(average_mat)
    corr_mat = torch.mm(torch.transpose(data, 0, 1),data)
    for i in range(len(imp_roi)):
        corr_mat[i,i] = 0
    corr_mat_z = corr_mat.arctanh()
    corr_mat_z = corr_mat_z.data.numpy()
    corr_mat = corr_mat.data.numpy()
    return corr_mat, corr_mat_z

In [5]:
# Connectivity matric all 141 ROI's
fmri_paper_sub_FC = {} 
fmri_paper_sub_FC_Z = {} 
for sub_id in subject_ids:
    corr_mat_later, corr_mat_later_z = extract_conn_hist(sub_id,imp_roi, data_dir, data_dir1)
    fmri_paper_sub_FC[sub_id] = corr_mat_later
    fmri_paper_sub_FC_Z[sub_id] = corr_mat_later_z
    print(sub_id)
    
with open('data/FC_Hist_avg/fmri_paper_sub_FC_NoScale.pickle', 'wb') as handle:
    pickle.dump(fmri_paper_sub_FC, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/FC_Hist_avg/fmri_paper_sub_FC_Z_NoScale.pickle', 'wb') as handle:
    pickle.dump(fmri_paper_sub_FC_Z, handle, protocol=pickle.HIGHEST_PROTOCOL)        

BG_04_01
BG_05_01
BG_06_01
BG_08_01
BG_09_01
BG_11_01
BG_14_01
BG_15_01
BG_16_01
BG_17_01
BG_18_01
BG_20_01
BG_22_01
BG_23_01
BG_24_01
BG_25_01
BG_26_01
BG_27_01
BG_28_01
BG_30_01
BG_32_01
BG_34_01
BG_35_01
BG_36_01
BG_37_01
BG_39_01
BG_40_01
BG_41_01
BG_42_01
BG_43_01
BG_44_01
BG_46_01
BG_47_01
BG_48_01
BG_49_01
BG_50_01
BG_51_01
BG_52_01
BG_53_01
BG_54_01
BG_55_01
BG_57_01
BG_58_01
BG_59_01
BG_62_01
BG_64_01
BG_65_01
BG_68_01
BG_71_01
BG_72_01
BG_73_01
BG_77_01
BG_78_01
BG_82_01
BG_83_01
BG_84_01
BG_85_01
BG_86_01
BG_89_01
BG_90_01
BG_92_01
BG_95_01
BG_96_01
BG_97_01
BG_99_01
BG_100_01
BG_102_01
BG_103_01
BG_104_01
BG_105_01
BG_107_01
BG_109_01
BG_110_01
BG_111_01
BG_112_01
BG_114_01
BG_115_01
BG_117_01
BG_118_01
BG_119_01
BG_122_01
BG_123_01
BG_124_01
BG_125_01
BG_126_01
BG_127_01
BG_129_01
BG_130_01
BG_131_01
BG_04_02
BG_05_02
BG_06_02
BG_08_02
BG_09_02
BG_11_02
BG_14_02
BG_15_02
BG_16_02
BG_17_02
BG_18_02
BG_20_02
BG_22_02
BG_23_02
BG_24_02
BG_25_02
BG_26_02
BG_27_02
BG_28_02
BG_3

In [6]:
# Connectivity matric only Saliance Network
loc_vec = np.zeros(len(aseg_2009_saliance_network), dtype = int)
for i in range(len(aseg_2009_saliance_network)):
    loc_vec[i] = np.where(imp_roi == aseg_2009_saliance_network[i])[0][0]
# print(loc_vec)
fmri_paper_sub_FC_SN = {}
fmri_paper_sub_FC_Z_SN = {}
for sub_id in subject_ids:
    fmri_paper_sub_FC_SN[sub_id] = np.zeros((len(aseg_2009_saliance_network),len(aseg_2009_saliance_network)))
    fmri_paper_sub_FC_Z_SN[sub_id] = np.zeros((len(aseg_2009_saliance_network),len(aseg_2009_saliance_network)))

for i in range(len(loc_vec)):
    for j in range(i,len(loc_vec)):
        for sub_id in subject_ids:
            fmri_paper_sub_FC_SN[sub_id][i,j] = fmri_paper_sub_FC[sub_id][loc_vec[i],loc_vec[j]]
            fmri_paper_sub_FC_SN[sub_id][j,i] = fmri_paper_sub_FC[sub_id][loc_vec[i],loc_vec[j]]
            fmri_paper_sub_FC_Z_SN[sub_id][i,j] = fmri_paper_sub_FC_Z[sub_id][loc_vec[i],loc_vec[j]]
            fmri_paper_sub_FC_Z_SN[sub_id][j,i] = fmri_paper_sub_FC_Z[sub_id][loc_vec[i],loc_vec[j]]
            
with open('data/FC_Hist_avg/fmri_paper_sub_FC_NoScale_SN.pickle', 'wb') as handle:
    pickle.dump(fmri_paper_sub_FC_SN, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/FC_Hist_avg/fmri_paper_sub_FC_Z_NoScale_SN.pickle', 'wb') as handle:
    pickle.dump(fmri_paper_sub_FC_Z_SN, handle, protocol=pickle.HIGHEST_PROTOCOL)             